<a href="https://colab.research.google.com/github/palmcorp/AOSP-ICSAce/blob/master/Copy_of_PROD_CP_Port_YOLO_2_Colab_117.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
VERSION = "10.0"
RELEASE="52"
MISSION = " Port of workstation CP Yolov8_np to Colab to generte Yolo_np"
MISSION+= "\n and estblish check points"
MISSION+="\n test NO data inerts"
MISSION+="\n Test RUN Evlutor"
MISSION+="\n recognize Q-Score"
MISSION+="\n evaluating Cycle Detecgor"
print ("version: ", VERSION, " \n M I  S S S O N", MISSION)
USE_MASTER = True # use ai/videos

version:  10.0  
 M I  S S S O N  Port of workstation CP Yolov8_np to Colab to generte Yolo_np
 and estblish check points
 test NO data inerts
 Test RUN Evlutor
 recognize Q-Score
 evaluating Cycle Detecgor


In [ ]:
import os, pdb
print ("Mount GOOGLE  Drive")
from google.colab import drive
DRIV=os.path.join ("/content/drive/MyDrive/CardiacEchos/Videos")
if os.path.exists (DRIV):
  print (DRIV, " exists, skipping")
  AI=os.path.join ("/content/AI/Videos")
  os.makedirs(AI, exist_ok=True) # just in case
else:
  drive.mount('/content/drive')
print ("go to Mydrive/CardiacEchos/Videos    ")
big_vid=os.path.join ("/content/drive/MyDrive/CardiacEchos/Videos")
LST_VIDS=os.listdir(big_vid)
print (len (LST_VIDS), " videos identifid")
print ("MAPPING TO MASTER ")
from distutils.dir_util import copy_tree
L_now =os.listdir("/content/AI/Videos")
if len (L_now)==10029:
    print ("already have ", len (L_now),"videos , skippig copy")
else:
  print ("Starting copy to AI")
  copy_tree("/content/drive/MyDrive/CardiacEchos/Videos", "/content/AI/Videos")
print ("\n DONe COPYING")
master_vid= os.path.join ("/content/AI/Videos")
LST_VIDS=os.listdir(master_vid)
print (len (LST_VIDS), " videos copied")

Mount GOOGLE  Drive
/content/drive/MyDrive/CardiacEchos/Videos  exists, skipping
go to Mydrive/CardiacEchos/Videos    
10029  videos identifid
MAPPING TO MASTER 
Starting copy to AI

 DONe COPYING
10029  videos copied


In [ ]:
print ("stage I; import python modules")
!pip install ultralytics

stage I; import python modules
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.4/709.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
print ("need GPU")
!invidia-smi


need GPU
/bin/bash: line 1: invidia-smi: command not found


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan  5 10:19:02 2024

@author: palm
"""

# wokin colab class
##########################################
##  CARDIAC PATTENS, INC.               ##
##                                      ##
##  YOLOv8 From Colab                  ##
##  p petronelli                        ##
##  01/05/24                            ##
##                                      ##
##  Class to instantiate and work Yolov8
##  for echos   DEMO
##                                      ##
##                                      ##
#######(c) Cardiac Patterns, Inc.  #######
########2023-2024  All rightes reserved.#######
##########################################
## change history
## ver   date    eng    action
## 00   01/05/24  plp    based on working colab
## 03   01/24/24plp      added no data inseets
##
##
###########################################
from ultralytics import SAM
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import numpy as np
import datetime
#import cv2
import os, pdb
import pickle as pkl
import time
###

class YOLOV8 :
    def __init__(self, ROOT_DIR):
        self.VERSION = "13"  # included zero data entries Specil edition
        self.FROM = "https://docs.ultralytics.com/models/yolov8/"
        self.BibTeX="@software {yolov8_ultralytics, url={https://github.comultralygtics/ultrlytics"
        now = datetime.datetime.now()
        Stamp = now.strftime ("%d-%B")
        self.CREATED = Stamp
        self.ECHO = False
        self.TEST = False
        self.YOLO_STATUS = False
        self.MODEL = None
        self.MODEL_NAME="yolov8x.pt" # stanard
        self.W_DIR ="" # none
        self.ROOT = ROOT_DIR # eerython falls from this
        WFP = os.path.join (ROOT_DIR, "WEIGHTS")
        self.WEIGHTS=WFP
        self.yolo_np =np.array([" "," ",0.1,"",0.2,0,"X"]) # (TAG,L1,C1,L2, C2,Q, X)
        self.yolo_np_default = np.array([" "," ",0.1,"",0.2,0,"X"])
        #DATA _STATUS (X)  0=uninitialized, 1= data intitialized (no AI ) 2=Current values (may iclude AI)
        #=
        self.SHAPE =self.yolo_np.shape
        self.STD   ="undefined"
        self.VID_DIR = self.STD
        self.VID_NP  = self.STD
        ## for saving results
        self.PICKEL_DIR = self.STD
        self.LAST_PICKEL_NAME =self.STD
        self.YOLO_NP = self.STD   # forml oe
        self.LAST_DATA_PICKLE_NAME=self.STD
        self.DATA_PKL_DATE= self.STD
        self.MISSION="special cola edition"


    def _config (self):
        cfg="\n YOLOV8 class: V_"+str (self.VERSION)+"\n ROOT DIR: "+str(self.ROOT)
        cfg+="\n Sub dir structue: \n  WEIGHTS (pretrined weights)\n  W_DIR (weights directory) \n  MODEL"
        cfg+="\n MODEL: "+str(self.MODEL_NAME) +"  model status: "+str(self.YOLO_STATUS)
        cfg+="\n HyperParameters: "+"imgsz=1024,show=False, show_labels=False, show_conf=False, save_conf=False,save_frames=False"
        cfg+="\n DATA STATUS (X) VALUES "+"(0=uninitialized, 1= data intitialized (no AI ) 2=Current values (may iclude AI))"
        cfg+="\n Created "+str(self.CREATED)
        cfg+="\n AVI DIR: "+str(self.VID_DIR)
        cfg+="\n ECHO: " +str(self.ECHO)+"  TEST: "+str(self.TEST)
        cfg+="\n Format of yolo_np: [TAG, Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE,yx]- NEEDS UPDATING"
        cfg+="\n PICKLE STATUS data pickel: " +str(self.LAST_DATA_PICKLE_NAME)+" date "+str(self.DATA_PKL_DATE)
        return cfg

    def _start_model (self):
        if self.ECHO: print ("stating model with model=YOLO(", self.MODEL)
        if self.TEST: pdb.set_trace()
        model=YOLO(self.MODEL_NAME)
        if self.ECHO: print ("model=", type (model))
        self.MODEL= model # for later use
        self.YOLO_STATUS=True
        return True

    def _set_model_name (self, MODEL_NAME):
        if self.ECHO: print ("changing model namde from ", self.MODEL, ' to ', MODEL_NAME)
        self.MODEL_NAME = MODEL_NAME
        return

    def _set_TEST(self, VALUE):
        if isinstance (VALUE, bool):
            self.TEST = VALUE
            return
        else: print ("Nota boolean ", VALUE)

    def _set_ECHO (self, VALUE):
            if isinstance (VALUE, bool):
                self.ECHO = VALUE
                return
            else: print ("Nota boolean ", VALUE)
            return

    def _get_empty_YOLOv8_np(self):
        if self.ECHO: print ("creting empty yyolv8")
        yolo_np = self.yolo_np # this is sthe efault
        return yolo_np


    def _create_np(self):
        if self.TEST: pdb.set_trace()
        TAG="ABC345"
        DIAG1="Mild"
        CONF1= 0.0
        DIAG2="Severe"
        CONF2=0.0
        Q=0
        YX=""
        TPL = (TAG, DIAG1,CONF1,DIAG2,CONF2,Q,YX)
        self.yolo_np =  np.array([TPL])  # null first element
        return True

    def _set_vid_dir (self, VID_DIR):
        if self.TEST: pdb.set_trace()
        if not os.path.exists (VID_DIR):
            print (VID_DIR, " does not eist")
            return False
        self.VID_DIR = VID_DIR
        return True

    def _set_vid_class(self, VID_CLASS):
        if self.ECHO: print ("setting vid clas in Yolo class")
        self.VID_CLASS = V=VID_CLASS
        #vid_np = V._get_vid_,np(self.VID_DIR)
        _,vid_np = V._get_vid_np()
        self.VID_NP = vid_np
        return

    def _get_yolo_np (self):
        if self.TEST:pdb.set_trace()
        return self.yolo_np, True

    def _one_row (self, TAG,L1,C1,L2,C2,Q,X):
        #format  [TAG, Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE,yx]"
        print ("ONE ROW not implemented")
        if self.TEST:pdb.set_trace()
        one = np.array([TAG, L1,C1,L2, C2, Q, X])
        return one

    def _exists(self, TAG):
        if self.TEST: pdb.set_trace()
        ntags = len (self.yolo_np)
        if ntags ==0: return False
        else:
            if ntags==1: return False
            for row in range (ntags-1):
                one_row =self.yolo_np[row]
                tg= one_row[0]
                if tg==TAG:
                    return True
        return False

    def _get_tag_values (self, TAG):
        if self.TEST: pdb.set_trace()
        ntags = len (self.yolo_np)
        if ntags ==0: return False
        else:
            for row in range (ntags-1):
                one_row =self.yolo_np[row]
                tg= one_row[0]
                if tg==TAG:
                    return True, one_row
            return False, np.empty (self.SHAPE)

    def _add_tag(self,TAG  ):
        return self._do_one_tag(TAG) # reurns OK + yolo_np

    # return values from Yolo_np
    def _Get_YOLO_Values (self,TAG):
        print ("retrieving ", TAG, "from yolo_np")
        OK,YOLO_ROW = self._get_tag_values (TAG)# reurns one row
        if not OK:
            if self.ECHO :print("some error in _Get_YOLO_Values")
            if self. TEST: pdb.set_trace()
            return False , ()   # tmpey

        #format  [TAG, Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE,yx]"
        TAG, Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE,yx =YOLO_ROW # up=npck
        TPL=(TAG,Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE, yx)
        if self.ECHO: print ("YPL VALUES: ",TPL)
        return True, TPL

    def _do_one_tag(self,TAG):
        if self.TEST: pdb.set_trace()
        #get vid path from TAG
        OK,TUPL=self.VID_CLASS._get_tag_values (TAG)
        if not OK:
            print ("cant find ",  TAG)
            return False
        TagNo,TAG,VID_NAME, VID_FULL_PATH, X = TUPL
        #TAG_FP = os.path.join (self.AVI_DIR, TAG)
        VID_FP = VID_FULL_PATH
        if not os.path.exists (VID_FP):
            print (VID_FP, "not exists")
            return False
        if not self.YOLO_STATUS:
            print  ("model not initilizd ERROR")
            return False

        model=self.MODEL
        Xresult=model.predict (VID_FP,save=False, imgsz=1024,show=False, show_labels=False, show_conf=False, save_conf=False,save_frames=False)  # seve frames!
        nR= len(Xresult)
        #find highest conf
        # conf = SUM(occurances * conf)/occurances
        # from   Imgsz for predicg is only one numbr from:https://stackoverflow.com/questions/76739392/yolov8-imgsz-in-predict
        SAVE_NAME = str(TAG)+"_"+"Result"+str(self.VERSION) #each sve ha TAG
        self._pickel_result ( Xresult, SAVE_NAME, MODE="save")

        TPLS=()
        SCORE_DICT=self._unpack_results(Xresult)
        #TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS)
        SD=SCORE_DICT
        #dict INCLUDES results!
        # TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS) # omit result
        # add to np
        #self.yolo_np
        # make one row
        # add to existing
        #unpack SD
        # for ONE TAG
        L1,C1, L2, C2 = self._calculate_w_ave (SD)  # SD are the totls or all frmes in the VID
        #one reult FOR EACH frame !
        Q=0
        X=""
        one_np = self._one_row(TAG, L1, C1, L2, C2, Q, X)  # ON ROE
        if self.TEST:pdb.set_trace()
        #yolo_np=np.vstack([[self.yolo_np],[one_np]])
        X=self.yolo_np
        Y=one_np
        yolo_np=np.vstack([X,Y])

        ''' skip
        add = np.array ([])
        for X in Xresult:
            conf1=X.conf
            lbl1 = X.label
            TPL=(lbl1,conf1)
            L1=""
            C1=0.0
            L2=""
            C2=0.0
            Q=0
            X="X"
            one_np = self._one_row(TAG, L1, C1, L2, C2, Q, X)
            yolo_np=np.vstack(self.yolo_np,one_np)
            '''
        if self.ECHO:print ("Done with video: ", TAG)

        if self.TEST:pdb.set_trace()
        self.YOLO_NP= yolo_np
        self.yolo_np = yolo_np
        return True

    def _set_PICKEL_DIR (self, PD):
        if not os.path.exists (PD):
            print (PD, " doesnt exist")
            pdb.set_trace()
        self.PICKEL_DIR=PD
        return
    ## save results in a pickel for later analysis
    def _pickel_result (self, Result, PICKLE_NAME, MODE):
        if self.PICKEL_DIR == self.STD:
            print (" pickel dir NOT initialized, pleae call _set_PICKEL_dir")
            return
        if MODE=="save":
            now = datetime.datetime.now()
            Stamp = now.strftime ("%d-%B")
            PD= self.PICKEL_DIR
            RESULT_PKL_NAME = PICKLE_NAME+"_"+str(Stamp)+".pkl"
            self.LAST_PICKLE_NAME = RESULT_PKL_NAME
            # check paths
            OK = True
            FPP= os.path.join (self.PICKEL_DIR, RESULT_PKL_NAME)
            if self.ECHO: print ("REsult pickel saved at", FPP)
            try:
                pkl.dump (Result, open (FPP, "wb"), protocol=pkl.HIGHEST_PROTOCOL)
                '''
                self.LAST_PICKEL_NAME = SAVE_NAME # only save if valid
                self.LAST_PICKEL_PATH = FPP
                self.VIDEO_STATUS = True
                self.VIDEO_NP_STATUS=True  # bogh re gtrue
                '''
            except Exception as EX:
                print ("Exce[topm ", EX)
                pdb.set_trace()
                OK=False
            return
        if MODE=="load":
            #print ("not iplemented")
        #if MODE=="load":
            try:
                if not os.path.isfile (self.PICKEL_DIR):
                    print (self.PICKEL_DIR, " not a valid pth to file - call _set_pickl_dir")
                    return False, ()
                RESTORE_PICKEL_NAME=PICKLE_NAME
                #FULL_PATH = os.path.join (self.LAST_CSV_FILE_PATH, self.LAST_PICKEL_NAME)
                #fileobject =  open (FULL_PATH, "rb")
                FULL_PICKLE_PATH =os.path.join (self.PICLEL_DIR, RESTORE_PICKEL_NAME)
                if self.ECHO:print ("retrieving pickle ", FULL_PICKLE_PATH)
                fileobject =  open (FULL_PICKLE_PATH , "rb") # NOTE FULL PATH
                #triage_array=pkl.load(fileobject)
                Result   = pkl.load(fileobject)  # must be same name as saved# must be same as saved
                fileobject.close ()
                #self.VIDEO_NP = VIDEO_NP# save
                #self.video_np = VIDEO_NP
                timestr = time.strftime("%Y%m%d-%H%M%S")
                self.PICKLE_LOAD_TIME = timestr
                #self.Triage_np = self.triage_array
                #self.video_np = VIDEO_NP # as loaded
               # self.VIDEO_NP = video_np
                #return True,triage_array
                #self.VIDEO_NP_STATUS= True # usable
               #self.VIDEO_STATUS = True
                self.LAST_PICKEL_NAME=RESTORE_PICKEL_NAME
                return True, Result  # not self as loaded
            except Exception as EX:
               print ("Error loading ",FULL_PICKLE_PATH ,"exception: ", EX)
               return False,() # empty ara

            return OK, Result

    def _load_results(self, PICKLE_NAME):
        Result=None  # at loagd
        XResult =self. _pickel_result (Result, PICKLE_NAME, MODE="load")

        return XResult

    def _test_Xresult (self, Xresult):
        for result in Xresult:
           SD=self._unpack_results (result)
           #TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS,result) # mighgt all be ZEORO ddin a;; result
         # HERE
           TAG="FE!F5"
           LABEL1="Mild"
           CONF1= 0.8
           LABEL2="Severe"
           CONF2= 0.
           Q=0
           X="expansion"
           SPL = (TAG, LABEL1, CONF1, LABEL2, CONF2,Q, X)
        return SPL

    def _calculate_w_ave (self,SD):
        nrow = len (SD)
        # sum by label
        ST={}  # empty dit
        LD=cm_plot_labels = ["Endocarditis","Mild","Non-Visible","Normal","Prosthetic","Severe"]
        # # TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS) # omit result
        for i in range (len(LD)):  # initialize
            ST[LD[[1]]]=0
        ALLCONF=0
        for row in range (nrow-1):
            Class1=row[1]
            conf1 = row[2]
            NAME1=row[5]
            x=ST[NAME1]
            nx = x + conf1
            ST[NAME1]=nx  # new x
            ALLCONF+=conf1
        # normalize
        if ALLCONF ==0:
          print ("zero total, exiting")
          LABEL=LABEL2="UNKWN"
          CONF= CONF2=0.0
          return LABEL, CONF, LABEL2,CONF2

        for i in len(ST):
            This= ST[LD[i]]
            newthis = This/ALLCONF  # normalized
            ST[LD[i]]=newthis
        print (ST)
        MAX = max(ST, key=ST.get)
        key=ST.get(max(ST, key=ST.get))
        CONF = ST.value[key]
        LABEL = LD[key]
        LABEL2=""
        CONF2=0

        return LABEL, CONF, LABEL2,CONF2


    def _get_model_arch (self):
        if not self.YOLO_STATUS :
            print ("Model NOT initialized")
            return "uninicialized"
        return str(self.MODEL)

    def PrepImage (self, IMMAGE_NAME, IMAGE_DIR):
        #make larger

        print ("prep Iag NOT Implemented")
        if self.TEST:pdb.set_trace()
        return

    def _load_yolo (self, MODEL_PT , WEIGHTS_FULL_PATH):
        if self.TEST: pdb.set_trace()
        OK=True
        MODELS = ("yolov8x.pt", "yolov8m.pt", "yolov8l.pt", "yolov8n.pt", "yolov8n.pt") # just the size
        if not MODEL_PT in MODELS :
            print (" Unrcognized YOLO Model : ", MODEL_PT)
            OK = False
        WFPP = WEIGHTS_FULL_PATH
        model=YOLO(MODEL_PT)
        self.MODEL_NAME = MODEL_PT  # pretrine model


        if not  os.path.exists (WFPP):
            print ("weights not a path:", WFPP)
            OK=False
        model = YOLO (WFPP)  # Normally 'yolovx.pt  # add weighgts
        self.YOLO_STATUS = True # can use the model
        self.MODEL = model
        self.WEIGHTS=WFPP  # note overrides automatic

        return OK,model

   # def Load_Pretrained (self, PRE_TRAIN):
    #   # load retraine into model
    # MOVED TO COLAB

    def _unpack_results (self, RESULTS):
        #global TK
        if self.TEST: pdb.set_trace()
        if self.ECHO:
            L_OUT = True
            #TK.L ("_unpack_results", "X")
        else :L_OUT=False
        SCORE_DICT =SD={}   #indexed y class
        N_FRAME=0 # identify this frame
        XN="Undef"
        KEEP="UNDEF"  # label to keep
        for result in RESULTS:
            # set defaulgts
            CONF2=0
            NAME2=XN
            CLASS2=XN
            CLASS1=XN
            CONF1=0
            NAME1=XN
            ANY=0
            #XN="Undef"
            BX=result.boxes.xywhn  # box with xywh format but normalized, (N, 4)
            if len(BX)==0: BX=0
            else:
                ANY+=1
                BX=BX.tolist()

            CONF=result.boxes.conf   # confidence score, (N, 1)
            if len(CONF)==0: CONF1=0
            else:
                ANY+=1
                nconf=len(CONF)
                #CONF=CONF.item()
                if nconf==1:
                    CONF1=CONF.item()
                    CONF2=0
                else:
                    CONF=CONF.tolist()
                    CONF2=CONF[1]  # HERE
                    CONF1=CONF[0]   # just the first


            CLASS=result.boxes.cls    # cls, (N, 1)
            if len(CLASS)==0:CLASS1=XN
            else:
                ANY+=1
                if len (CLASS )==1:
                    CLASS1=CLASS.item()
                    NAME1 =result.names[CLASS1]
                else:
                    CLASS=CLASS.tolist()
                    if self.TEST:pdb.set_trace()
                    #print ("picking first class")
                    NAME1 = result.names [CLASS[0]]#jut th highest
                    R=result
                    nclass=len(CLASS)
                    #print (R.names[int(CLASS[0])])# R.names[int(CLASS[1])])
                    #print (CONF[0], CONF[1])
                    rec="For "+str(R.names[int(CLASS[0])])+ " conf: "+str(CONF[0])+ "/"+str(CONF[1])
                    print (rec) # put this in TRACK
                    CLASS1= CLASS[0]  # keep the first
                    if nclass>1:
                        CLASS2=CLASS[1]
                        NAME2=result.names [CLASS[1]]
                        print (R.names[int(CLASS[1])])
                    else:
                        CLASS2=XN
                        NAME2=XN



            PROBS=result.probs     # cls prob, (num_class, )
            if PROBS==None: PROBS = 0
            else:
                ANY+=1
                PROBS=PROBS.tolist()

            if ANY==0:
                    # didnt find any
                    if L_OUT: print ("couldnt find any for ", N_FRAME, " keeping: ",  KEEP)
                    N_FRAME+=1
                    continue
            else:
                TAG=self._tag_from_path(result.path)
                #TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS,result) # mighgt all be ZEORO ddin a;; result
                TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME1,NAME2,N_FRAME, BX,PROBS) # omit result
                SD[N_FRAME]=TPL  # add to dict.  KEY is FRAME #
                N_FRAME+=1
                if L_OUT:
                    if KEEP!= NAME1:
                        arg="changing KEEP from: "+str(KEEP)+" to " +str(NAME1)+" with conf: "+str(CONF1)
                        print (arg)
                        #TK.T(arg)
                        KEEP=NAME1
                    STPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME1,NAME2,N_FRAME)
                    print (STPL)
                    #TK.T(str(STPL))

        nCONF=3
        print ("number of frames : ", N_FRAME, " for this video")
        if self.TEST: pdb.set_trace()
        # AD RESULT TO DICT
        return SD

    def _tag_from_path(self, PATH) :
        if self.ECHO: print ("")
        P=PATH
        dot = P.find(".")
        TAG = P[dot-5:dot]
        return TAG


         ### updatged form colab
         ##TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS,result)

    def _calculate_w_ave (self,SD):
        LTEST=False
        nrow = len (SD)
        # sum by label
        ST={}  # empty dict for STATISTICS
        LD=cm_plot_labels = ["Endocarditis","Mild","Non-Visible","Normal","Prosthetic","Severe"]
        # # TPL = (TAG,CLASS1,CONF1, CLASS2,CONF2, NAME,NAME2,N_FRAME, BX,PROBS) # omit result
        for i in range (len(LD)):  # initialize
            IND=LD[i]
            ST[IND]=0
        #pdb.set_trace()
        ALLCONF=0
        if LTEST: pdb.set_trace()
        for Xrow in  range (nrow-1):# ffix
          # check if key exists
            if not Xrow in SD:continue
            row=SD[Xrow]
            TAG=row[0]
            Class1=row[1]
            conf1 = row[2]
            NAME1=row[5]
            x=ST[NAME1]
            nx = x + conf1
            ST[NAME1]=nx  # new x
            ALLCONF+=conf1
        # normalize
        if self.ECHO: print ("Total cnf is ", ALLCONF)
        for i in range(len(ST)):
            This= ST[LD[i]]
            if ALLCONF==0:
              print ("Avoiding div zero set  ALLCONF =1")
              ALLCONF=1
            newthis = This/ALLCONF  # normalized
            ST[LD[i]]=newthis

        V,K =max(ST.items(), key=lambda k: k[1])
        #pdb.set_trace()
        CONF = K  #ST.value[key]
        LABEL = V  #LD[key]
        #pdb.set_trace()
        LB=sorted (ST, key = ST.get)[-2]

        CONF2=ST[LB]
        LABEL2=LB
        #CONF2=0

        return LABEL, CONF, LABEL2,CONF2



    def Predict1 (self, IMAGE, IMAGE_DIR):
        IFP=os.path.join (IMAGE_DIR, IMAGE)
        if not os.path.isfile(IFP):
            print (IFP, "not an exiting  fiels")
            return False
        if not self.YOLO_STATUS :
            print ("Model NOT INITIALIZED")
            return False

        ## do prediction
        model = self.MODEL
        results = model(IFP)

        return
    def PredictALl (self, SOURCE_DIR):
        S=SOURCE_DIR
        model=self.MODEL
        OK=True
        if not os.path.exists (S):
            print (S, " doesnt exist")
            OK = False
        nS = len (os.listdir(S))
        print (nS, "images found")
        results = R=model(S)
        nR=len(R)
        print (nR, " rsutulst")
        return OK, R

    def PredictVID(self,VID_DIR):
        V=VID_DIR
        OK=True
        model = self.MODEL
        if not os.path.exists (V):
            print (V, " does not xist")
            OK = False
            return OK
        results=R=model (V)
        nR=len (R)
        print (nR, " reults found")
        return OK, R

    def _set_W_dir (self,TEMP_DIR):
        T=TEMP_DIR
        print ("using ", T, " as working  frame dir")

        if not os.path.exists (T):
           print (T, " does not exist")
           return False
        self.W__DIR =T
        return True

    def _DUMP_R (self, RESULTS):
          R=RESULTS
          LABELS=[]
          LBEL_TXT=""
          boxes = R.boxes
          for box in boxes:
            c = box.cls
            lbl=R.names[int(c)]
            if lbl !="":
              LABEL=lbl
              LABELS.append(lbl)
              LBEL_TXT+=str(lbl)+", "
          rLBLS=LBEL_TXT
        # exgacg fields
          rIM=str(R.orig_img)+"\n"  #raw ddata , skip
          rK=str(R._keys)+"\n"
          rB=str(R.boxes)+"\n"
          rKP=str(R.keypoints)+"\n"
          rMK=str(R.masks)+"\n"
          rN =str(R.names)+"\n"
          rO=str(R.obb)+"\n"
          rPBS=str(R.probs)+"\n"
          rSD=str(R.save_dir)+"\n"


          conf=R.boxes.conf
          if len(conf)==0: # cantmake to a scalar
              xconf=0
          else:
              xconf=conf.item() # get actual value of this tensor
          #confidence = conf.item()
          line2="\n"
          conf=R.boxes.conf
          #confidence = conf.item()
          confidence = conf # dont conver to alar if im ==
          rCF="conf="+str(conf)+"\n"

          if len(conf)==0: # cantmake to a scalar
             xconf=0
          else:
              xconf=conf.item() # get actual value of this tensor

          rCF="conf="+str(conf)+"\n"
          # FN = file name
          line2+="LABEL:"+str(LABEL)+" conf: "+str (xconf)
          if self.ECHO : print (line2)



          SEP="==="
          line="\n"+SEP+rK+rB+rKP+rMK+rO+rPBS+rN+rLBLS+"\n"+rSD+rCF+line2+SEP

          return LABEL, xconf, line
    # pickles a data area in the pickel dir
    def _pickel_YOLO_NP (self, DATA, PICKLE_NAME, MODE):
     if self.PICKEL_DIR == self.STD:
         print (" pickel dir NOT initialized, pleae call _set_PICKEL_dir")
         return
     if MODE=="save":
         now = datetime.datetime.now()
         Stamp = now.strftime ("%d-%B")
         PD= self.PICKEL_DIR
         DATA_PKL_NAME = PICKLE_NAME+"_"+str(Stamp)+".pkl"
         self.LAST_DATA_PICKLE_NAME = DATA_PKL_NAME
         # check paths
         OK = True
         FPD= os.path.join (self.PICKEL_DIR, DATA_PKL_NAME)
         if self.ECHO: print ("REsult pickel saved at", FPD)
         DATA=self.YOLO_NP
         if self.ECHO:print ("savind YOLO_NP")
         try:
             pkl.dump (DATA, open (FPD, "wb"), protocol=pkl.HIGHEST_PROTOCOL)
             self.DATA_PKL_STATUS = True
             self.DATA_PKL_DATE   = Stamp
         except Exception as EX:
             print ("Exce[topm ", EX)
             pdb.set_trace()
             OK=False
         return  True
     if MODE=="load":
         #print ("not iplemented")
     #if MODE=="load":
         try:
             PICK_FP=os.path.join (self.PICKEL_DIR, PICKLE_NAME)
             if not os.path.isfile (PICK_FP):
                 print (self.PICKEL_DIR, " not a valid pth to file - call _set_PICKLE_DIR")
                 return False, ()
             RESTORE_DATA_NAME=PICKLE_NAME
             #FULL_PATH = os.path.join (self.LAST_CSV_FILE_PATH, self.LAST_PICKEL_NAME)
             #fileobject =  open (FULL_PATH, "rb")
             FULL_PICKLE_PATH =os.path.join (self.PICKEL_DIR,  RESTORE_DATA_NAME)
             if self.ECHO:print ("retrieving pickle ", FULL_PICKLE_PATH)
             fileobject =  open (FULL_PICKLE_PATH , "rb") # NOTE FULL PATH
             #triage_array=pkl.load(fileobject)
             Data   = pkl.load(fileobject)  # must be same name as saved# must be same as saved
             fileobject.close ()
             #self.VIDEO_NP = VIDEO_NP# save
             #self.video_np = VIDEO_NP
             timestr = time.strftime("%Y%m%d-%H%M%S")
             self.DATA_PKL_DATE = timestr
             self.YOLO_NP = Data
             self.yolo_np = Data
             self.DATA_PKL_NAME=PICKLE_NAME
             return True, Data  # not self as loaded
         except Exception as EX:
            print ("Error loading ",FULL_PICKLE_PATH ,"exception: ", EX)
            return False,() # empty ara

         return OK, Data
     ##########################3
     ## Null data updates (v10)#
     ###########################
    def _add_data_row (self, TUPL):
       if self.TEST: pdb.set_trace()
       print ("NOT IPLEMENTED")
       # adds a ne row o Yolo_NP  with default data
       pdb.set_trace()

    def _update_row (self, TAG, **kwags):
        print ("updage field in a row")
        pdb.set_trace()





In [ ]:
# from https://stackoverflow.com/questions/73362781/qt-qpa-xcb-could-not-connect-to-display-when-using-yolov4-custom-functions
print ("create virtual wiindos for colab")
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

create virtual wiindos for colab
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 32 not upgraded.
Need to get 7,814 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5bu

In [ ]:
!Runtime -> Change run time type -> GPU,TPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

/bin/bash: line 1: Runtime: command not found
Thu Feb  1 19:33:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+---------------------

In [ ]:
print ("Add libraries")
from ultralytics import SAM
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import numpy as np
import datetime
import cv2
import os, pdb


Add libraries


In [ ]:
print ("Creating all directories JUST FOR COLA")
DIRS=("MODEL", "WEIGHTS","STORE",'VIDEOS',"FRAMES", "RESULTS", "LIBS", "TRIAGE","CHECK_POINTS","REPORTS", "BLIND_STUDY")
ROOT=os.path.join ("/content/")
print ("ROOT IS ", ROOT)
def MDIRS (ROOT, TUPL):
  for D in TUPL:
    fp=os.path.join (ROOT, D)
    print ("creating 1st level dir: from", ROOT, D)
    os.makedirs(fp, exist_ok=True)
  print ("done with dirs")
  return True
# call from main?
ok= MDIRS (ROOT, DIRS)



Creating all directories JUST FOR COLA
ROOT IS  /content/
creating 1st level dir: from /content/ MODEL
creating 1st level dir: from /content/ WEIGHTS
creating 1st level dir: from /content/ STORE
creating 1st level dir: from /content/ VIDEOS
creating 1st level dir: from /content/ FRAMES
creating 1st level dir: from /content/ RESULTS
creating 1st level dir: from /content/ LIBS
creating 1st level dir: from /content/ TRIAGE
creating 1st level dir: from /content/ CHECK_POINTS
creating 1st level dir: from /content/ REPORTS
creating 1st level dir: from /content/ BLIND_STUDY
done with dirs


In [ ]:
def CHECK_IT (PATH, NAME):
  que="Upload "+str(NAME)+ " to"+str(PATH)
  xx= input (que)
  if xx=="": return True
  print ("you typed ", xx)
  return False

In [ ]:
print ("migrate code to colab")
xx=input ("upload best.pt to weights then cr")
if xx=="": print ("next")
xx= ("upload libs to LIBS")
if xx=="": print ("next")
xx=input ("upload 60+ videos to VIDEOS")
if xx=="":print ("next")
xx=input("load libs into LIBS")
if xx=="":print ("next")
xx=input ("upload 3 (three) to TRIAGE"+str("TRIAGE/CARDIAC_TRIAGE0.220230824-110623.pkl and ALL_MASTER_TRIAGE_071223_v01.csv"))
if xx=="": print ("next")
xx=input ("upload to STORE: Cardic_Patterns_Video_Lib_02_20240113-143422.pkl")
if xx=="": print ("next")
ok = CHECK_IT ("/content/BLIND_STUDY/","Blind_Study_consolidation_MASTER_1205223v03.csv")


migrate code to colab
upload best.pt to weights then cr
next
upload 60+ videos to VIDEOS
next
load libs into LIBS
next
upload 3 (three) to TRIAGETRIAGE/CARDIAC_TRIAGE0.220230824-110623.pkl and ALL_MASTER_TRIAGE_071223_v01.csv
next
upload to STORE: Cardic_Patterns_Video_Lib_02_20240113-143422.pkl
next
Upload Blind_Study_consolidation_MASTER_1205223v03.csv to/content/BLIND_STUDY/


In [ ]:
# HEADER
CODE_BASE="blind_rpt v262  from 1/18/24"
print (" HEADER from" , CODE_BASE)
print (" setCHDIR TO  lib root")
LIB_ROOT=os.path.join ("/content/LIBS")
NOW=os.getcwd() # whree are we now
os.chdir(LIB_ROOT)

from CardiacLIB import Report
from  CardiacRESLib import RESULT, TAG_FROM_VID, FRAME_NO
from CardiacTriage  import PROCESS_TRIAGE
from CardiacUtilLIB import CHECK_UP, PATH_UP, PICK_ONE,PICK_ONEX,_TRACK, TAG_FROM_PATH
from CardiacGTLIB import F_1
from CardiacBlindLIB import BLIND_STUDY
from CardiacValues  import cm_plot_labels
import time
####


from CardiacYOLOlib import YOLOV8
from CardiacVIDlib  import VIDEO

from datetime import date
import datetime
# update
import os, pdb

#CONFIGUGION PARAMETER
global MARGIN
global TK  # for tracking
xNOW=os.getcwd() # whree are we now
NOW=os.path.join ("/content")
print ('CHANGED CWD BACK TO', NOW, "FROM ",xNOW)
pdb.set_trace()
os.chdir(NOW)
##########################################
##  CARDIAC PATTERNS,INC.               ##
##                                      ##
## BLIND Study  REPORT GENRRATOR        ##
##
##  p petronelli                        ##cm_plot_labels
##  11/20/23                            ##
##                                      ##
##  Contents:                           ##
#      uses SCI-KIT LEARN               ##
##     uses Read Ground truth from  lib ##
##  THis code creates the final summay rpt ##
##                                      ##
##                                      ##
#######(c) Crdiac Patterns     , Inc.   #######
########2020-2024  All rightes reserved.#######
##########################################
##  UPDATES
##  Ver   Date   Isseue                        Eng
## 38    11/25/23
## 40    11/2723    Add summry epotrt generation plp
##50     1/6/24   plp  Convert AI from YOLO8
## 260   1/17/24  plp  Added pickel save and load
##       1/16/24  plp  testing on Colab
########################

 HEADER from blind_rpt v262  from 1/18/24
 setCHDIR TO  lib root



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



CHANGED CWD BACK TO /content FROM  /content/LIBS
--Return--
None
> <ipython-input-11-38f771c53461>(34)<cell line: 34>()
     32 NOW=os.path.join ("/content")
     33 print ('CHANGED CWD BACK TO', NOW, "FROM ",xNOW)
---> 34 pdb.set_trace()
     35 os.chdir(NOW)
     36 ##########################################

ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



In [ ]:
xx=input ("unzip vids from VIDEOS.zip at CHECK_POINTS= cr")
if xx=="":
  #!unzip CHECK_POINTS\VIDEOS.zip

  ZIP_NAME = "VIDEOS.zip"
  ZIP_PATH = os.path.join ("/content/CHECK_POINTS", ZIP_NAME)
  ZIP_END_POINT = os.path.join ("/content/VIDEOS")
  print ("unzipping from ", ZIP_PATH, " to ", ZIP_END_POINT)
  if not os.path.isfile(ZIP_PATH):
    print (ZIP_PATH, " not a file")
  if not os.path.exists (ZIP_END_POINT):
    print ("not exist ", ZIP_END_POINT)
  !unzip /content/CHECK_POINTS/VIDEOS.zip  -d /content/VIDEOS
  LISTV=os.listdir(ZIP_END_POINT)
  print ("stored ", len(LISTV), " videos")

unzip vids from VIDEOS.zip at CHECK_POINTS= crno


In [ ]:
# PART TWO - U T I L I T I E S
print ("Part Two U T I L I T I E S")
# fclass = F_1(OPTIONS)
# To procress F_1.Load_CSV_Ground_Truth (self, CSV_PATH, CSV_NAME)
   # from CP-yolo_detect_and_count-v0115.py
def LOAD_TRIAGE_PKL (TRIAGE_MASTER_REPORT_NAME, TRIAGE_MASTER_REPORT_DIR, AVI_PATH):
         OK=True
         ECHO = True
         TEST=False # at v61
         if ECHO: print (" LOAD TRIAGE WITH: ",TRIAGE_MASTER_REPORT_NAME, TRIAGE_MASTER_REPORT_DIR, AVI_PATH)
         if TEST: pdb.set_trace()
         try:
             PTC = PROCESS_TRIAGE(TRIAGE_MASTER_REPORT_NAME, TRIAGE_MASTER_REPORT_DIR, AVI_PATH)

             PTC._set_test(False)
             PTC._set_ECHO((True))
             #HISTORICAL_PATH = PL=os.path.join ("C:\\","DEV4","PROCESS_TRIAG
             PL = TRIAGE_MASTER_REPORT_DIR # use current not hisgorical conv
             HISTORICAL_PATH =PL
             PKL_LIST = os.listdir(PL)
             np,pick= PICK_ONEX (PKL_LIST, "pkl")

             PKL_NAME= "CARDIAC_TRIAGE0.220230824-110623.pkl"
             if pick == PKL_NAME:
                 print ("OK")
             else:
                 xx= input ("Aare you sure? not stanard")
                 pdb.set_trace()
             MODE = "load"
             PICKEL_NAME =LAST_PICKEL_NAME = PKL_NAME
             LAST_PICKEL_PATH = HISTORICAL_PATH  #conv
             LAST_PICKEL_PATH= os.path.join (PL)   # must NOt be full pagh
             PTC._set_test(False)
             #pdb.set_trace()
             PTC._set_last_pickel_data (LAST_PICKEL_NAME, LAST_PICKEL_PATH)
             OK,TRIAGE_ARRAY_NP = PTC._pickel_(PICKEL_NAME, MODE)
         except Exception as EX:
             print (" Exception in Load triage; ", EX)
             pdb.set_trace()
             OK = False

         return OK,PTC

def GET_TRIAGE (TRIAGE_DIR,VID_DIR):
          # set up irectgories
          TRIAGE_MASTER_REPORT_NAME="ALL_MASTER_TRIAGE_071223_v01.csv"
          #TRIAGE_MASTER_REPORT_DIR=os.path.join ("D:\\","MASTER_RUN") # conv
          TRIAGE_MASTER_REPORT_DIR=TRIAGE_DIR
          #AVI_PATH=os.path.join ("C:\\","AI")
          AVI_PATH= VID_DIR  # conv
          OK,PTC= LOAD_TRIAGE_PKL (TRIAGE_MASTER_REPORT_NAME, TRIAGE_MASTER_REPORT_DIR, AVI_PATH)
          if not OK:
             print ("problem getting array")
             pdb.set_trace()
          return PTC  # return the class
'''
class Yolov8 :
    def __init__(self) :
      print ("initializin yolo lass" )
    def GET_CLASS(self):
        yolov8=""
        return yolov8

def GET_YOLOV8_CLASS ():
    YOLOV8_CLASS = Yolov8()
    return YOLOV8_CLASS
 '''

def Get_AI (TAG, TRIAGE_CLASS, YOLOV8_CLASS):
    print ("processing TAG: ", TAG)
    pdb.set_trace()

    ## UPdate with AI from Yoov8
    return


def list_vid(FP):
    #=(FP):
    lst=os.listdir(FP)
    print (len(lst), "v in", FP)
    return lst

#def Read_csv (FP_IN):
    # read in csv ground truth / blind annoations rom sharol

def  Read_IN (FP_IN):
    fp=open (FP_IN)
    START=False
    IN_LIST = [] #
    nl = 0
    with open (FP_IN) as fp:
        for line in fp:
            print (nl,"  ", line)
            nl+=1
            if "Sharol," in line:
                 # skip
                 print ("skipping", line) #break
                 break
            if not START:
                    if "Attachments" in line: START=True
            else:
                # started, read 0X9DB79DF0A269227.mp4; 0X9E3D09429F6B852.mp4;
                if ".mp4;"in line:
                    flds=line.split(";")
                    print (flds)
                    f1=flds[0]
                    f2=flds[1]
                    IN_LIST.append (f1)
                    IN_LIST.append (f2)

    print ("end of in list of , ", len(IN_LIST))
    return IN_LIST

def _Get_TAG_values ( TAG, Triage_np):
    # triage array
    # assume data is clean

    #FRAMES = np_row[1]

  #  if TAG.islower():TAG.upper()
   # TAG=self._clean_spaces(TAG)
    TEST = False
    #pdb.set_trace()
    if TEST: pdb.set_trace()
    print (" fetching TAG:", TAG)
    FOUND = False
    NP_data = Triage_np
    lnp = len (NP_data)-1  # fim zero
    print ("lnp is ", lnp)
    T1=20
    TX=0
    for i in range (lnp):
        TX+=1
        np_row = NP_data[i]
        tag= np_row[2] # tags should alway be upper
        T1-=1
        INCREMENT = False
        if INCREMENT:
            if T1==0:
                print (np_row)
                x=input ("GO or x, or new number")
                if x=="x":
                    print ("quitting at ", TX)
                    pdb.set_trace("stop here")
                    return True, []    #self._get_row_values (np_row)
                if x=="":
                    T1=20
                else :
                    nn= int (x)
                    if isinstance (x, int):
                        T1=nn

        if tag==TAG:
            FOUND = True
            NUM   = np_row[0]
            FRAMES = np_row[1]
            TAG= np_row[2] # tags should alway be upper
            #if TAG.islower():TAG.upper()
            # TAG=self._clean_spaces(TAG)
            print ("Found ", TAG, "at ", TX, " \n ",np_row)

            TPL = np_row  # []# self._get_row_values(np_row)
            return FOUND,TPL
    print (TAG, "NOt found after ", TX)
    return False,() # epmpty
def SPELL_UP(LABEL):
    ECHO=True
    # check comon checkin erro
    STD=cm_plot_labels  #cm_plot_labels = ["Endocarditis","Mild","Non-Visible","Normal","Prosthetic","Severe"] #alphabetic
    COMMON_SPELLUP=CU={'Mil':'Mild', "Norml":"Normal", "Sever":"Severe", "Prothetic":"Prosthetic", "Prostetic":"Prosthetic","Stenosed":"Stenosis"} # dicionary
    if LABEL=="SPELL_DICT":
        return COMMON_SPELLUP
    if LABEL in CU.keys():
        NEW_LABEL = CU[LABEL]
        if ECHO: print (" updated ", LABEL, " to ", NEW_LABEL)
        return NEW_LABEL
    else: return LABEL



def READ_SHROL_EVAL (CONSOLIDATD_CSV_NAME, PATH):
    #global TK
    args = str(CONSOLIDATD_CSV_NAME)+ ' |'+ str (PATH)
    print (args)
    #TK.L ("READ SHAOL EVAL", args)
    print (" Read in prepare d echo ecaluations")
    TEST = False
    CN=CONSOLIDATD_CSV_NAME
    CP=PATH
    MODULE = "READ SHAROL EVAL"
    if TEST: pdb.set_trace()
    # use read ground truth  raw data
    # create class
    OPTIONS = (False, False)  #   Track, Test
    F1= F_1 (OPTIONS)
    print ("reading in consolicated csv file: ", CN, " AT ", CP)
    CSV_PATH = CP
    CSV_NAME = CN
    Echo_data = F1.Load_CSV_Ground_Truth (CSV_PATH, CSV_NAME)
    if TEST: pdb.set_trace()
    # returns a list of TAGs?
    MT=False
    if MT:
        MILD_TAGS=MT=[('D4F5F','Severe'),('FB24F','Mild'),('58F10','Mild'),('ACFE8','Mild'), \
                      ('7CCE4','Mild'),('C5714','Mild'),('541A0','Mild'),('87BA6','Mild'), \
                          ('0D042','Mild'),('27D5C','Omit')]  # (TAG , AI)  THEE Ae HSOL RFS
        return MT
    print ("Not fixed list retueinve list of TAG,SHAROL")
    SH_list =[]
    for E in Echo_data:
        tag = E[0]
        sharol= E[2]
        Qscore= E[4]
        if "omit" in E:
            print ("found OMIT set Q to 1")
            Qscore = 1
            if TEST: pdb.set_trace()
        SH_list.append ((tag,sharol,Qscore))
    args= str (len(SH_list))+ " sharol cases foudn"
    #TK.T(args)
    print (args)
    return SH_list , Echo_data
    #return Echo_data

def BUILD_BLIND_REPORT ():
    print ( "creatint blind summary report")
    ##################
    ## set up blind report ##  11/27
    ##################
    TODAY=datetime.now().strftime("%Y%m%d-%H%M%S")
    COMMON_SUFFEX =CS=str(VERSION)+"_"+str(TODAY)+".txt"
    BLND_NAME = "Cardiac_Patterns_Blind_Study_Results_Report_"+  CS
    RPT_BASE  = os.path.join ("D:\\","CP_BLIND_STUDY","PROCESSED_REPORTS")
   # d_list = os.listdir(DATE_BASE)
   #print (" pick a date")
    #nx,pick = PICK_ONE(d_list)
    #RPT_DATE = pick
    #RPT_NAME ="Cardiac_Patterns_Blind_Study__"+CS   #str(VERSION)+"_"+str(RPT_DATE)+".txt"
   # SRT_NAME ="Cardiac_Patterns_Blind_Score__"+CS   #str(VERSION)+"_"+str(RPT_DATE)+".txt"
    # do areports
   # P_RPT_DIR =os.path.join (RPT_BASE,"report")
   # S_RPT_DIR =os.path.join (RPT_BASE, "score")
    B_RPT_DIR = os.path.join (RPT_BASE, "summary")  # this is the glind report

    REPORT_NAME = RPT_NAME= BLND_NAME
    P_RPT_DIR   = RPT_BASE
    #RPTD  =BLIND_STUDY(P_RPT_DIR, REPORT_NAME)
    #REPORT_NAME = SRT_NAME  # for score sheet
    #SRTD  =BLIND_STUDY(S_RPT_DIR, REPORT_NAME)
    BRPT = BLIND_STUDY(B_RPT_DIR, BLND_NAME)
    return BRPT


def CHECK_AGREEMENT (AI,CONF,LBL2, CONF2, SHAROL, TAG):
    global TK
    MODULE="Check _Ageement"
    ECHO=True
    print ("checking agreemente begween sources FOR ai:", TAG)
    arg="TAG: "+str(TAG)+" AI:"+str (AI)+ " conf: "+str (CONF)+" LBL2\CONF: "+str(LBL2)+"|"+str(CONF2)+ " Sharol: "+str(SHAROL)
    TK.L ( "CHECK GREEMENT", arg)
    if ECHO: print (arg)
    Equivocal =EQ= 0  # adjudiates in close calls
    global MARGIN
    # compare two strings using Case insenstitive
    # remove white spaces
    #if SHAROL == 'Stenosed':
    ECHO= True
    TEST=False
    A=AI.strip()
    S=SHAROL.strip()
    S=SPELL_UP(S)
    SHAROL=SPELL_UP(SHAROL) # check both
    EQ_MARGIN = MARGIN *100  # percentagaes!#0.45  # WITHIN 10 % MEANS EQ# adjustd for v91
    if  A.casefold()==S.casefold():
       arg= "found equivalents "+str( AI)+ "|"+str( SHAROL)
       TK.T(arg)
       EQ=0
       TK.T("")
       return True, EQ
    elif  SHAROL == 'Stenosed' and AI=="Severe":
        arg="found Stenosed == Severe"
        TK.T(arg)
        return True, EQ
    elif  SHAROL == 'Stenosis' and AI=="Severe":
        arg="found Stenosis and Severe"
        TK.T(arg)
        return True, EQ
    elif SHAROL == "Prosthetic" and AI=="Severe":
        arg="found Prosthetic and Severe "
        TK.T(arg)
        EQ=0
        return True, EQ
        # consider AI at 45 and 23  with professional gees 2nd label
    else:
        #pdb.set_trace() # exmine data
        B=LBL2.strip()
        C=float(CONF.replace("%",""))
        C2=float(CONF2.replace("%",""))
        TOLERANCE=0  # to avoid probl
        arg="In second test"+str(C)+"|"+str(C2)
        TK.T(arg)
        if B.casefold()==S.casefold(): # TRY SECOND labl declare eq
            # edge case
            # a hit if conf2 within EQ_MARGINE of conf gthen ecalre EQ
            #TOLERANCE = CONF-CONF2
            TOLERANCE = (C-C2)/100 # normalized leady percentge
            TOLERANCE = abs((C-C2)) # just in case
            if TOLERANCE <=EQ_MARGIN :
                EQ=1
            if ECHO: print ("boundaries : ",TOLERANCE )
        if ECHO:
            arg= "Equivocal Case: LBL1, CON1"+str( AI)+"|"+str(CONF)+"  lbl2/conf2"+str( LBL2)+"|"+str( CONF2)+" SHAROL: "+str( SHAROL)
            arg+= " Tolerance: "+str( TOLERANCE)
            TK.T(arg)
        if TEST: pdb.set_trace()
        if EQ==1:
            arg="return TRUE "+str(EQ)
            TK.T(arg)
            return True, EQ
    if ECHO: print ("No match on first or second for ", SHAROL, " at ", TAG)
    arg ="No match on first or second for "+str( SHAROL) + " at "+str( TAG)
    TK.T(arg)
    return False , EQ # no match on first or second



Part Two U T I L I T I E S


In [ ]:
# PART THREE M A I N
print ("PART THREE - M A I N")
VERSION = "0.265"
MISSION = " ADD AI EVALUATION TO REPORT & TRACKING & YOLOV8 predictions @ v 250"
MISSION+="\n v260 is working!! c v 261 has sharol limit to shorten running"
MISSION+="\n porting to colap and genrealize configurations"
NEW_MISSION= " Read list of tag + ai + sharol as CSV  add Equivocal NOW SELEECT LIST"

def main (CONFIG):
    global MARGIN
    global TK
    YOLO_FORMAT = True
    print ("using yolo format: ", YOLO_FORMAT)
    print ("set breakpoints now")
    pdb.set_trace()
    CTEST=False  # conv
    DPTH=os.path.join (os.getcwd())# default
    # note VID_DIR in a config is root echos, my not work here
    ROOT, TRIAGE_DIR, VID_DIR, LIBS,WEIGHTS, STORE, REPORTS, CHECK_POINT, BLIND_STUDY = UNPACK_CONFIG(CONFIG)
    print ("\n CONFIGURATION:",ROOT, TRIAGE_DIR, VID_DIR,LIBS, WEIGHTS, STORE, REPORTS,CHECK_POINT, BLIND_STUDY)
    MARGIN = 0.45  # FROM ANALYSIS
    if CTEST: pdb.set_trace()
    TEST=False
    CHECK=True  # CHEOCN AGREEMENT
    MODULE = "Main"
    print ("\n ENTERING M A I N with ", ROOT)
    MISSING_TAGS=[] # list of missing TAGS
    SPELL_DICT = SPELL_UP("SPELL_DICT")
    print ("spelling corrections: \n ITEM CORRECTION\n")
    for k,v in SPELL_DICT.items():
        print (k,"\t",v)

    xx=input ("generated score report from email cr=no")
    if xx=="":
        READ_DATA = False # check data to read
    else:
        READ_DATA = True
    today = date.today()
    # CHECK ON Q SCORE
    print ("\nEqui-val Margin is ",MARGIN, " \n" )
    if READ_DATA:
        SRPT_DIR = os.path.join (os.getcwd())
        SNAME    = "Cardiac_Patterns_Blind_ECHOS_"+str(VERSION)+"_"+str(today)+".txt"
        SHD = " Evaluation Score Card_v"+str(VERSION)
        TT="\t"
        SFLDS = "NO"+TT+"TAG"+TT+"SHAROL"+TT+"FILE NAME"+TT+"DATE\n"
        short_rpt= SRPT = Report(SRPT_DIR, SNAME,SHD, SFLDS)
        IP= os.path.join ("D:\\","CP_BLIND_STUDY","PROCESSED_REPORTS")
        IP= os.path.join ("D:\\","CP_BLIND_STUDY","RE-ISSUE","REPORT")
        IPN="Next_BLIND_study_incement.txt"
        IPN="sharol 12-5 _data.txt"
        LST = os.listdir(IP)
        np,pick = PICK_ONEX(LST, ".txt")
        if pick != IPN:
            print ("wrong link? ", IPN, " vs ", pick)
            pdb.set_trace()
        else:
             IPN=pick
             print ("selecting ", pick)
        FP_IN = os.path.join (IP, IPN)
        IN_LIST = Read_IN (FP_IN)
        print ("\n Contents of ", IPN)
        SPACE = TT+str(today)+"\n\n"
        for i in range (len(IN_LIST)):
            print ("[",i,"] ",IN_LIST[i])
            LN=IN_LIST[i]
            TAG=TAG_FROM_VID(LN)
            line = TT+str(TAG)+TT+"____________"+TT+str(LN)+SPACE
            SRPT.addL(line)
        SRPT.endFile("End of list Nov 28")
        print ("look in ", SRPT_DIR)
        pdb.set_trace()
    today = date.today()
    if TEST:pdb.set_trace()  # conv
    T_CLASS= T=   GET_TRIAGE (TRIAGE_DIR, VID_DIR)
    Triage_np = T._get_Triage_np()
    #pdb.set_trace()
    TEST_TAGS = False
    if TEST_TAGS:
        print ("TESTING TAG RETEAL")
        TEST_TAGS =TT= [('3368B','PROSTETIC'),('7E925','MILD'),('24933','MILD'),('2A231','NORMAL'),\
                     ('DC01C','STENOSED'),('EE3A2','NORMAL'),('5EA5B','NORMAL'), ('EOE45','STENOSED'),\
                         ('8F990','N0RMAL'),('2559F','STENOSED')]
        MILD_TAGS=MT=[('D4F5F','Severe'),('FB24F','Mild'),('58F10','Mild'),('ACFE8','Mild'), \
                      ('7CCE4','Mild'),('C5714','Mild'),('541A0','Mild'),('87BA6','Mild'), \
                          ('0D042','Mild'),('27D5C','Omit')]  # (TAG , AI)  THEE Ae HSOL RFS
        print (" from 6/23 MILD study ", MT)
        XT=TT+MT
        for tl in XT:
             TAG=tl[0]
             LBL=tl[1]
             OK, TPL = _Get_TAG_values (TAG, Triage_np) # use local function
             if not OK:
                 print ( 'problem at ', TAG)
                 pdb.set_trace()
             else:
                 print (" at ", TAG, "sharol: " , LBL,"AI:", TPL)
        print ("end of internal test on TAGS")
    #pdb.set_trace()
    today = date.today()
    DATE = today
    ## dont need this
    '''
    FP = os.path.join (ROOT,"videos","28-November")  # conv
    L=list_vid(FP)
    '''
    STOP=False
    if STOP: pdb.set_trace()
    #self, DIR, NAME, HEADER, FIELDS,TYPE=0, ADD=False):
    RPT_DIR = REPORTS # from config os.path.join ("D:\CP_BLIND_STUDY\PROCESSED_REPORTS")
    NAME="Cardiac_Patterns_Consolidatd_Blind_Study_"+str (DATE)+"_"+str(VERSION)+"_"+str(RELEASE)+".txt"
    HD = "BLIND STUDY VIDEOS for\t"+str(VERSION)+" RELEASE: "+str(RELEASE)
    HD+="\n AI Source:  Yolov8 \n MODEL: yolov8x\n"
    TT="\t"
    FLDS = "No"+TT+"TAG"+TT+"AI"+TT+"CONF" +TT+"LABEL2"+TT+"CONF2"+TT
    FLDS+="SHAROL"+TT+"FILE_NAME"+TT+"AGREEMENT"+TT+"EQUIVOCAL\n"
    # for YOLO
    FLDS ="No"+TT+"TAG"+TT+"GND_TRUTH"+TT+"AI"+TT+"CONF" +TT+"LABEL2"+" / "+"CONF2"
    FLDS+=TT+"AGREEMENT"+TT+"EQUIVOCAL\n"
    RPT = Report(RPT_DIR, NAME,HD, FLDS)
    print ("report: ", NAME, " at ", RPT_DIR)
    LTEST=False  # local test
    if LTEST:
        FLIST = list_vid(FP)
        FLIST =IN_LIST
        print ("processing ", FP_IN )
    CSV_NAME="Blind_Study_consolidation_MASTER_112623v03.csv"
    #PATH   = os.path.join ("C:\\","Backup-OFFSITE","0000CardiacPatterns","F1-STUDY","F-1Study","F-1DATA")
    # updagte faor blind reporaat
    #PATH  = os.path.join ("C:\\","Backup-OFFSITE","0000CardiacPatterns","BLIND_Study","CONSOLIDATION")
    PATH   = BLIND_STUDY
    if LTEST:pdb.set_trace() # temp
    CSV_LIST = os.listdir (PATH)
    print ("pick consolidated source from ", PATH)
    num, pick = PICK_ONEX(CSV_LIST, ".csv")
    if not isinstance (num, int):
        print ("cant")
        pdb.set_trace()
    else:
        CSV_NAME = pick
    # whre is the READ CSV
    SHAROL_LIST, Echo_data = READ_SHROL_EVAL(CSV_NAME, PATH)
    # set up tracking
    TRACK_FILE = "Cardiac_Patterns_TRACK_RECORD_"+str(VERSION)+"_"+str(RELEASE)+".txt"
    TRACK_DIR  = os.path.join (RPT_DIR,"track")
    TK=_TRACK (TRACK_FILE,TRACK_DIR)
    ARGS = " Track recored from "+str(today)+" version: "+str(VERSION)
    TK.L("MAIN", ARGS)

    #for f in FLIST :
    A_Count = 0# READ_SHROL_EVAL number of agreements
    R_Count = 0 # number of records processes
    X_Count = 0 # number of disagreements
    M_Count = 0 # number of marginal deciions
    N_Count = 0 # numer of NO AI
    NValid  = 0 # count of valid cases
    LTEST = False
    TEST=False
    #####
    print ("Engaging YOLOV8 with root:", ROOT)
    if TEST: pdb.set_trace()
    #A#VI_DIR =VD= os.path.join ("C:\\", "AI", "videos"  )
    AVI_DIR =VD= VID_DIR  # conv FROM COFIG
    #PICKEL_DIR =PD=os.path.join (ROOT, "STORE")
    PICKEL_DIR = PD=STORE

    print ("initializing video class & pickel dir:\n", VD,"\n :", PD)
    #VID_CLASS = V=VIDEO (AVI_DIR)
    VID_CLASS = V = VIDEO (VID_DIR) # thi is active videos not ROOT vids which may not benecessary
    V._set_TEST(False)
    OK=V._get_vid_np_status()   #_get_vid_np_status
    if OK:
     VID_np=V._get_vid_np()
    else: print ("VID_NP Not initilized, creating new array [ may take a while], or request pickel")
    xx=input ("cr = go with pickel")
    if xx!="":
        VID_STAT,VID_np=V._create_vid_np(VD)
        if not VID_STAT:
            print("couldn't create vid_np (empty)")
    ## seg up pickle

        if not os.path.exists (PD):
            os.makedirs(PD)
            print ("creating ", PD)
        print ( "pickel version of video is at ",PD)
        V._set_pickel_dir(PD)  # this is OK
        V._set_vid_dir (VID_DIR)  # neessary aftger convert
        LAST_PICKEL_PATH = "unconfigured"
        LAST_PICKEL_NAME = "unconfigured"
        LAST_PICKEL_DIR  = "unconfigured"
        Plist = os.listdir(PD)  # list of pickeled vids
        # check VID_STAT
        #if not VID_STAT:# its neer oin to  be TRue
        print ("Initilizing Video NP array from pickle")
        PICKEL_NAME="Cardic_Patterns_Video_Lib_"  # save fills in gthe rest
        xx=input ("vidstat:"+" create new pickenl only,vid np n,= cr=retrieve pickle")
        print ("NOT IMPLEMEntD")
        pdb.set_trace()
    if xx=="n":
            print ("FORCE create new pickel?")
            FORCE=True
            nx=-1
            VID_STAT,VID_np=V._create_video_np(VD)
            if not VID_STAT:
                print ("still no vide_np etter quit")
                pdb.set_trace()
            OK, FULL_PICKEL_PATH = V._pickel_(PICKEL_NAME, 'save',FORCE)
            print ("Full pickel path is: ", FULL_PICKEL_PATH)
            if not OK:
                print ("some error")
                pdb.set_trace()
            '''
        elif xx=="S":
            nx,pick = PICK_ONEX(Plist, "pkl")
        else:
            VID_DIR = os.path.join (AVI_DIR)
            VID_NP=V._just_np( AVI_DIR)
            pdb.set_trace()
             '''
    elif xx=="":
    # SELECT EXIXITING PICKLE
        Plist=os.listdir(PICKEL_DIR)
        if not os.path.exists (PICKEL_DIR):
            print ("doesnt exist: ", PD)
            pdb.set_trace()
        nx,pick = PICK_ONEX(Plist, "pkl")
        if nx<0:  # not pick or none
            print ("No saved pickels or force new :create new pickle")
            FORCE=True
            PICKEL_NAME="Cardic_Patterns_Video_Lib"
            V._set_ECHO(True)
            V._set_TEST(True)

            #PICKEL_DIR = PD=os.path.join (ROOT,"STORE") # alray
            #V._set_pickel_dir(PD)
            PPP=os.path.join (PD,PICKEL_NAME) # might be  problem if forced
            #V._set_pickel_path(PPP)  # full path
            '''
            #VID_NP = V._create_vid_np(AVI_DIR)
            if not V._set_pickel_data(PICKEL_NAME, PD):
                print ("pickel error")
                pdb.set_trace()
                '''
            OK, FULL_PICKEL_PATH = V._pickel_(PICKEL_NAME, 'load',FORCE)
            print ("Full pickel path is: ", FULL_PICKEL_PATH)
            if not OK:
                print ("some error")
                pdb.set_trace()
        else: # using existing pickel
            print (" exiting pkl, usng ",pick)
            PICKEL_NAME = pick
            PICKEL_PATH  =PP=os.path.join (PICKEL_DIR, pick)
            V._set_pickel_dir(PICKEL_DIR)
            V._set_pickel_path (PICKEL_PATH)  # FULLL PATH
            OK, VID_NP=V._pickel_(PICKEL_NAME, "load")
            #OK,NVID= V._create_vid_np(AVI_DIR)  # hould alredy be there fro m pickle
            if not V._get_VID_STATUS():
                print ("bad status after pickel load")
                pdb.set_trace()
            XVID=V._just_np(AVI_DIR)
            if not OK:
                print ("problem loading video np") #VID_NP2 = V._get_vid_np()
                pdb.set_trace()

    print ("status of vid: ", V._get_vid_np_status())

    #VID_np =V._get_vid_np()  # shoul alreaey hqe it
    OK,VID_np=V._create_vid_np(AVI_DIR)# conve.  creage vid np in curent en ieomn
    if not OK:
      print ("error in re-ceating vid_np")
      pdb.set_trace()  # conv upate
    VID_np=V._just_np(AVI_DIR)  # aove doesnt work
    LTEST=False
    if LTEST: pdb.set_trace()
    yolov=Y=YOLOV8(ROOT)
    Y._set_vid_dir(AVI_DIR)
    Y._set_vid_class(V)
    Y._set_TEST(False)
    Y._set_ECHO(True)
    Y._set_PICKEL_DIR (PD)
    MODELS = ("yolov8x.pt", "yolov8m.pt", "yolov8l.pt", "yolov8n.pt", "yolov8n.pt") # just the size
    MODEL_NAME="yolov8x.pt"
    #MODEL_NAME="yolov8l.pt"  # try smaller model NO
    WEIGHTS_NAME = "best.pt"
    #W_FP = WEIGHTS_NAME=os.path.join ("D:\\","CP_BLIND_STUDY","PROCESSED_REPORTS","MODEL", WEIGHTS_NAME)
    W_FP = Weights_Full_path = os.path.join (WEIGHTS, WEIGHTS_NAME) # fixed for conv
    OK, model= Y._load_yolo(MODEL_NAME, W_FP)
    Y._set_TEST(False)
    Ok=Y._create_np()
    OK=Y._set_vid_dir(AVI_DIR)
    XTEST=False
    if not OK:
        pdb.set_trace()
    if XTEST: pdb.set_trace()
    Y._set_TEST(False)
    Yolo_np,OK=Y._get_yolo_np() # geg entrie arrayhoule be empty at this point
    print (Y._config())
    if LTEST:pdb.set_trace()
    RESULT_PICKEL_PATH = RPD=os.path.join (ROOT, "RESULT")
    os.makedirs (RPD, exist_ok=True)
    Y._set_PICKEL_DIR(RPD)
    LATE=False # no late start
    xx=input ("limit ground truth = number or cr = no or s=skip this many  befor starting")
    if xx=="":
        print ("dont limit")
        SHAROL_LIMIT = False
        NSHAROL      = 10000 # EFECTIELYINFINITE
    elif xx=="s":
      print ("enter numberer to skip and later how many")
      xx=input ("how manty to skip?")
      try:
        nSkip = int (xx)
        NSKIP = nSkip
        xx= input ("then how many to try")
        nshrol= int (xx)
        SHAROL_LIMIT = True
        NSHAROL = nSkip+nshrol
        LATE = True
      except Exception  as EX:
          print ("some problem", EX)
          pdb.set_trace()
    else:
        try:
            nshrol= int (xx)
            SHAROL_LIMIT = True
            NSHAROL=nshrol
        except Exception as EX:
            print ("some problem: ", EX)
            pdb.set_trace()
    Nsharol=0
    ####
    ## STAGE I of MAIN
    ## scn for exiting TAGS in Yolo_Np
    ### add and perform  model if not there
    ######
    ECHO=True
    PTEST=False # checking on echo entries
    Nscip=0  # cound num r of skipped
    V._set_TEST(False) # so not too manuy igerruptions
    Q_LIST=[]  # lisgt of tags eliminated due to Quality score
    STOP=False
    if STOP: pdb.set_trace()  # check poit to exaind yool_np
    # sharol list format: (tag,sharol,Qscore)
    for S in SHAROL_LIST :
        # SHROL LISTtag,sharol,Qscore
        if SHAROL_LIMIT :print ("working on sharol no. ",Nsharol, " of ",NSHAROL)
        if LATE: # o lagte sgart
           if  Nsharol > NSKIP:  # then take rest seriously
               Nscip+=1
           else:
              Nsharol+=1 # keep counting
              continue
        TAG=S[0]
        if TAG=="TESTX": continue
        DIAG=S[1]
        Qscore=S[2]
        if Y._exists(TAG):
            print ("adding ", TAG, " to Yolo_p")
            if ECHO: print ('Ground truth list: ', S)
            rec="Adding TAG to yolo "+str(TAG)+ "whih sharol: "+ str(S)+ " and Q_Score: "+str(Qscore)
            TK.T(rec)
            if (Qscore!= 0) and(Qscore!=" "):
              print ("Qscoe NOT zero, Ommmitting", TAG, " is ",Qscore)
              QTUPL=(TAG, DIAG,Qscore )
              Q_LIST.append ((QTUPL))
              continue
            #Y._set_TEST(True)
            YOLO_NP_ONE_ROW= Y._get_tag_values (TAG) # TAG VALUES =([" "," ",0.1,"",0.2,0,"X"])
        else:
            rec=str(TAG)+ " does not exist in Yolo, runn model and add Gound Truth: "+str(S)
            if ECHO: print (rec)
            TK.T(rec)
            Y._set_TEST(False)  #check out conversion to colab
            OK =Y._add_tag(TAG)  #= DO ONE TAG, il el RUN THE MODEL
            if not OK:
              rec="Error return from add tag for "+str(TAG)+ " at echo# "+str(Nsharol)
              TK.T(rec)
              print ("\n ", rec)
            # retuns OK + single copye of Yolo_np
            print ("\n\n P A U S E for imwrite")
            if PTEST: pdb.set_trace()
        Nsharol+=1
        if SHAROL_LIMIT:
          if Nsharol>=NSHAROL: break

        #OK=Y._do_one_tag(TAG)
     # what's in YOLOV_NP?
    print ("end of sharol list  saving Results in pickle")
    #print ("HERE IS ERROR CHECK PICKEL STORE!!")
    # shrol list format: (<TAG>, <GROUND TRUTH>,<QCO.DE))
    #                       0        1           2
    if LTEST:pdb.set_trace() # TRaCE FROM HERE
    #Yolo_np, OK=Y._get_np()
    Y._set_TEST(False)  # conv gtesting
    Yolo_np, OK =Y._get_yolo_np()
    if not OK: pdb.set_trace()

        ########################
        ### YOLO NP IS now ready
        ### with TAG, AI-LABBE , ai_conf1, etc.
        #   Set up checkpoint save
        #######################
    CHECK_DIR = CHECK_POINT  # set by config really CHECKPOINTS
    TEST=False
    print (Y._config() )
    if TEST: pdb.set_trace()
    Y._set_TEST(False)  # after working
    #YOLO_PICKEL_DIR = YPD=os.path.join (ROOT, "STORE")
    YOLO_PICKEL_DIR = YPD=os.path.join (CHECK_POINT)
    print ("Setting check point dir to ", YPD)
    Y._set_PICKEL_DIR ( YPD)
    PICKLE_NAME = "CP_YOLOV8 "+str(VERSION)  # module adds time stmp
    Y._pickel_result (Yolo_np, PICKLE_NAME, MODE="save")  #NOT NOWbYES
    if LTEST:pdb.set_trace()
    Nsharol=0
    # SHAROL LIST FORMAT:
    #  TAG gROUN_trUTH, q-SCORE
    rec="\nStarting STAGE II process and write report\n"
    print (rec)
    TK.T(rec)
    SPT=True
    STOP=False
    if STOP: pdb.set_trace()
    for S in SHAROL_LIST:
        # get fp from TRIAGE
        #fp=os.path.join (FP,f)
        #TAG=TAG_FROM_VID (fp)
        rec="working on sharol no. "+str(Nsharol) + " of "+str(NSHAROL)
        TK.T(rec)
        if SHAROL_LIMIT :print (rec)
        if LATE: # o lagte sgart
           if  Nsharol > NSKIP:  # then take rest seriously
               Nscip+=1
           else:
              Nsharol+=1 # keep counting
              continue
        Nsharol+=1
        TAG = S[0]
        GTruth= S[1]
        rec="processing Sharol ["+str(Nsharol)+"] "+str(S)
        TK.T(rec)
        if SPT: print (rec)
        if LTEST: pdb.set_trace()
        Qscore = S[2] ##from zerro
        if '"' in TAG:
            if LTEST: print ('removing quote from ', TAG)
            TAG.replace('"','')
        line = TT+TAG
        # get AI diagnosis
        STOP=False
        #OK, TPL = Y._get_tag_values (TAG) # do we need this?
        if TAG =="TESTX" : continue
        if STOP: pdb.set_trace()
        OK, YPL =Y._Get_YOLO_Values( TAG)  # returns AILABEL, AICONF, AI2LABEL, AI2ONF
        if not OK:
          rec=str(TAG)+ "does not exist in Yolo_np skipping"
          print (rec)
          TK.T(rec)
          continue
        if STOP: pdb.set_trace()
        if len(YPL)==6:
            Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE, XY = YPL
            XTAG = TAG
        else:
            XTAG, Y_LABEL1, Y_CONF1, Y_LABEL2, Y_CONF2, Y_QSCORE, XY = YPL
            # USE DECODED LABELS FROM THIE POIN
        rec="\nYolo vvalues "+" for TAG:"+str( TAG) + " is "+str(YPL)
        TK.T(rec)
        if SPT:print (rec)
        #removed x
        # NOTE: YPL = ['4BC345' ,'Mild','0.9','Severe' ,'0.0' ,'0', '']
        # one row=one_row(TAG, L1, C1, L2, C2, Q, X)
        # NOTEE YOLO FORMAT HAs changed from erlier
        # NO fILE name in yolo format


        if LTEST : pdb.set_trace()
        NO_AI = False
        if not OK:
            if ECHO: print ("not Ok from get YLO VALUES AT ", TAG)
            FILE_NAME = "unk"
            CONF      = 0
        else:
          # note in YOLO NO file name.
          if YOLO_FORMAT:
            ECHO=True
            #FILE_NAME = TPL[3]
            FILE_NAME  = "Unspecified"
            #CONF      = TPL[6]
            #CONF      = YPL[2] # conv go YOLO
            AI= Y_LABEL1
            #pyCONF=Y_CONF1.item()
            #CONF = f"{pyCONF:.4f}"
            label2=Y_LABEL2
            #pyconf2=Y_CONF2.item()
            #conf2= f"{pyconf2:.4f}"
            CONF=Y_CONF1
            conf2=Y_CONF2
            SHAROL = GTruth  # s[2]
            rec="Settint up YOLO FORMAT AI:"+str(AI)+" conf: "+str(CONF)+"  lbl/conf2:"+str(label2)+"/"+str(conf2)+ " GT:"+str(SHAROL)
            TK.T(rec)
            if ECHO: print (rec)
          else:
            if ECHO: print ("none yoluo format")
            CONF      = TPL[6]
            FILE_NAME = TPL[3]
        if not OK:
            print ("cant get ai for ", TAG)
            AI="__________"
            NO_AI = True
            print ("ommitting ", TAG, ' no ai')
            MISSING_TAGS.append (TAG)
            N_Count +=1
            rec="cant get Ai for "+str(TAG)+"ommitting at "+str(N_Count)
            TK.T(rec)
            continue
        else:
          if YOLO_FORMAT:
            #AI=YPL[1]  # conv
            #CONF=YPL[2] # conv updage
            # alrey set up skip
            rec="Keeping yolo formlt"
            TK.T(rec)
          else:
            AI = TPL[5]
        if CHECK:
            #pdb.set_trace()
            rec=" At check for "+str(TAG)
            TK.T(rec)
            if YOLO_FORMAT:
              label2=Y_LABEL2    #YPL[3]
              conf2 =Y_CONF2 # YPL[4]
            else: # must be non yolo
              label2=TPL[7]
              conf2 = TPL[8]
            NValid+=1  # total valid cases
            SHAROL = S[1]  # FROM SHAROL LIST
            # ai SHOULD BE label
            AGR, EQ= CHECK_AGREEMENT (AI, CONF,label2, conf2,SHAROL, TAG)
            rec="CHecking for AI/CONF/L2/onf2/sharol/TAG: "+str(TAG)+" / "+str(AI)+" / "+str(CONF)+" / "+str(label2)+" / "+str(conf2)+" GT:"+str(SHAROL)
            TK.T(rec)
            if EQ==1: M_Count+=1
            R_Count +=1  # add up total recoreds
            if AGR:
                A_Count +=1
                CC="1"
                if EQ==1:
                    CC="Equivocal"
            else:
                CC="0"
                X_Count+=1  # Disagreements

        line+=TT+str(AI)+TT+str(CONF)+TT+str(label2)+TT+str(conf2)
        line+=TT+str(SHAROL)+TT+str(FILE_NAME)
        line+=TT+str(CC)#+"\n"
        ## usent YOLO FORMAT
        line = TT+TAG ### THI IS THE LINE
        XCONF= CONF  #f"{CONF:.4f}"
        xconf2= conf2  #f"{CONF2:.4f}"
        line+=TT+ str(SHAROL)
        line+=TT+str(AI)+TT+str(XCONF)+TT+str(label2)+" / "+str(xconf2)
        line+=TT+str(CC) +TT+str(Y_QSCORE)  #"\n"

        # add yolo line
        Yline="YOLOV8_"+str(TAG)+TT+Y_LABEL1+TT+ Y_CONF1+TT+ Y_LABEL2+TT+ Y_CONF2+TT+Y_QSCORE
        TK.T(line)
        TK.T(Yline)
        # output q list
        '''
        Qline = "Excluded TAG due to Q_score"
        for Q in Q_LIST: Qline+=str (Q)+", "
        RPT.addL(Qline)
        '''

        RPT.addL(line)
        #RPT.addL(Yline)
        # chek limit
        Nsharol+=1

        if SHAROL_LIMIT:
            if Nsharol==NSHAROL:
                print ("sharol limit of ", NSHAROL, " reached")
                break

    arg="calculating summary,look in"+str(RPT_DIR)
    print (arg)
    TK.T(arg)
    XTEST=False
    if XTEST: pdb.set_trace()
    sumL="SUMMARY \n\t TOTAL RECORDS"+TT+ "VALID CASES"+TT+"TOTAL AGREEMENTS "+TT+"DISAGREE"+TT+"RATIO"+TT+"EQUIVOCAl"+TT+"NO AI \n"
    sumL+=TT+str(R_Count)+ TT+str(NValid)+TT+str(A_Count)+TT+str(X_Count)+TT
    if NValid==0:
      NValid = 1
      print ("cant calculate percentae [DIV zero]")
      R=0
    else:
      R=f"{int(A_Count)/int(NValid):.2%}"
    sumL+=str (R)+TT
    sumL+=str(M_Count)+TT+str(N_Count)
    TK.T (sumL)
    RPT.addL(sumL)
    marginL ="\n\n "+"Equivocal Margine is "+str(MARGIN)
    RPT.addL(marginL)
    TK.T(marginL)
    print ("\n\n MISSING TAGS: ", MISSING_TAGS)
    mline ="\n Missing "+str (len(MISSING_TAGS))+" TAGs  for v"+str(VERSION)+ " as of "+ str(today)    +"\n"
    RPT.addL(mline)
    TK.T(mline)
    ETEST = False # check numbers f or v129
    if ETEST: pdb.set_trace()
    line = ""
    for i in range (len(MISSING_TAGS)) :    line+=str(MISSING_TAGS[i])  +",  "
    RPT.addL(line)
    TK.T(line)
    arg="end of report at "+str(RPT_DIR)
    print (arg)
    TK.T(arg)
    # output q list
    Qline = str(len(Q_LIST)) +"  Excluded TAGs due to Q_score "
    for Q in Q_LIST: Qline+=str (Q)+", "
    RPT.addL(Qline)
    RPT.endFile("END OF BLIND LIST")
    TK.E()
    if LTEST: pdb.set_trace()
    return

PART THREE - M A I N


In [ ]:
print ("settting up configurations")
C_CONFIG={"ROOT":os.path.join ("/content/")}
C_CONFIG["LIBS"]=os.path.join ("/content/LIBS")
C_CONFIG["WEIGHTS"]=os.path.join ("/content/WEIGHTS")
C_CONFIG["STORE"]=os.path.join ("/content/STORE")
C_CONFIG["VIDEOS"]=os.path.join ("/content/VIDEOS")
C_CONFIG["REPORTS"]=os.path.join ("/content/REPORTS")
C_CONFIG["TRIAGE"]=os.path.join ("/content/TRIAGE")
C_CONFIG["CHECK_POINT"]=os.path.join ("/content/CHECK_POINTS")
C_CONFIG["BLIND_STUDY"]=os.path.join ("/content/BLIND_STUDY")
if USE_MASTER:
  C_CONFIG["VIDEOS"]=os.path.join ("/content/AI/Videos")
  print ("\nUsing Master Video source: ", "/content/AI/Videos")
#pdb.set_trace()
'''
print (C_CONFIG)
ROOT=C_CONFIG["ROOT"]
LIBS=C_CONFIG["LIBS"]
VIDEOS=C_CONFIG["VIDEOS"]
print (ROOT,"\n", LIBS, VIDEOS)
'''
ROOT=os.path.join ("D:\\","CP_BLIND_STUDY",'PROCESSED_REPORTS')
A_CONFIG={"ROOT":ROOT}
A_CONFIG["LIBS"]=os.path.join (ROOT,"LIBS")
A_CONFIG["WEIGHTS"]=os.path.join (ROOT,"WEIGHTS")
A_CONFIG["STORE"]=os.path.join (ROOT,"STORE")
A_CONFIG["VIDEOS"]=os.path.join ("C:\\","AI")# NOT OFF OF ROOT
A_CONFIG['REPORTS']=os.path.join (ROOT,'report' )
A_CONFIG['TRIAGE']=os.path.join ("D:\\","MASTER_RUN" )# FROM DIFFERENT PROJECT
A_CONFIG['CHECK_POINT']=os.path.join (ROOT,"CHECK_POINTS" )
A_CONFIG['BLIND_STUDY']=os.path.join (ROOT,"BLIND_STUDY" )
BROOT=os.path.join ("E:\\","BLIND_STUDY")
B_CONFIG={"ROOT":BROOT}
B_CONFIG["LIBS"]=os.path.join (BROOT,"LIBS")
B_CONFIG["WEIGHTS"]=os.path.join (BROOT,"WEIGHTS")
B_CONFIG["STORE"]=os.path.join (BROOT,"STORE")
B_CONFIG["VIDEOS"]=os.path.join (BROOT,"VIDEOS")
B_CONFIG["VIDEOS"]=os.path.join (BROOT,"VIDEOS")
B_CONFIG["CHECK_POINT"]=os.path.join (BROOT,"CHECK_POINTS")
B_CONFIG["BLIND_STUDY"]=os.path.join (BROOT,"BLIND_STUDY")

settting up configurations

Using Master Video source:  /content/AI/Videos


In [ ]:
SYSTEM_A='a'
SYSTEM_B='b'
SYSTEM_C='c' # colab
PLATFORM="unknown"
def SELECT_PLATFORM():
  xx=input ("select platform a, b, or c =colab")
  O=xx.lower()
  if O=='a':
    PLATFORM = SYSTEM_A
    return A_CONFIG
  elif O=='b':
    PLATFORM=SYSTEM_B
    return PLATFORM
  elif O=='c':
   PLATFORM=SYSTEM_C
   PLATFORM="Colab"
   print ("P L A T F O R M=",PLATFORM)
   return C_CONFIG
  else:
    print ("unknown configuration, you typed", xx)
  return None




In [ ]:
print ("STARTING")
CONFIG=SELECT_PLATFORM()
print ("you selected for",PLATFORM, " with ", CONFIG)

STARTING
select platform a, b, or c =colabc
P L A T F O R M= Colab
you selected for unknown  with  {'ROOT': '/content/', 'LIBS': '/content/LIBS', 'WEIGHTS': '/content/WEIGHTS', 'STORE': '/content/STORE', 'VIDEOS': '/content/AI/Videos', 'REPORTS': '/content/REPORTS', 'TRIAGE': '/content/TRIAGE', 'CHECK_POINT': '/content/CHECK_POINTS', 'BLIND_STUDY': '/content/BLIND_STUDY'}


In [ ]:
def UNPACK_CONFIG(CONFIG):
  #pdb.set_trace()
  ROOT=CONFIG["ROOT"]
  VID_DIR=CONFIG["VIDEOS"]
  LIB_DIR=CONFIG["LIBS"]
  BEST_DIR=CONFIG["WEIGHTS"] # =  best.pt
  STORE   =CONFIG["STORE"]
  RPT_DIR =CONFIG["REPORTS"]
  TRIAGE_DIR=CONFIG["TRIAGE"]
  CHECK_POINT=CONFIG["CHECK_POINT"]
  BLIND_STUDY=CONFIG["BLIND_STUDY"]
  return ROOT, TRIAGE_DIR, VID_DIR, LIB_DIR,BEST_DIR, STORE, RPT_DIR,CHECK_POINT,BLIND_STUDY

In [ ]:
# parT four start
if __name__=='__main__':
          print (" from ", os.getcwd(), " VERSION", VERSION, " \n MISSION: ", MISSION)
          #pdb.set_trace()
          PROJ=  os.getcwd()  # We are here
          #CONFIG=ROOT, VID_DIR, LIB_DIR,BEST_DIR, STORE =SELECT_PLATFORM
          print ("ECHO CONFIG")
          print (CONFIG)
          main (CONFIG)
          print ("END of PROCESSING")
          pdb.set_trace()

 from  /content  VERSION 0.265  
 MISSION:   ADD AI EVALUATION TO REPORT & TRACKING & YOLOV8 predictions @ v 250
 v260 is working!! c v 261 has sharol limit to shorten running
 porting to colap and genrealize configurations
ECHO CONFIG
{'ROOT': '/content/', 'LIBS': '/content/LIBS', 'WEIGHTS': '/content/WEIGHTS', 'STORE': '/content/STORE', 'VIDEOS': '/content/AI/Videos', 'REPORTS': '/content/REPORTS', 'TRIAGE': '/content/TRIAGE', 'CHECK_POINT': '/content/CHECK_POINTS', 'BLIND_STUDY': '/content/BLIND_STUDY'}
using yolo format:  True
set breakpoints now
> <ipython-input-14-1d334503d7ee>(16)main()
     14     print ("set breakpoints now")
     15     pdb.set_trace()
---> 16     CTEST=False  # conv
     17     DPTH=os.path.join (os.getcwd())# default
     18     # note VID_DIR in a config is root echos, my not work here

ipdb> l 374
    369         else:
    370             rec=str(TAG)+ " does not exist in Yolo, runn model and add Gound Truth: "+str(S)
    371             if ECHO: print (r

100%|██████████| 131M/131M [00:01<00:00, 75.9MB/s]


Streaming output truncated to the last 5000 lines.
Severe

('B4030', 5.0, 0.6978257298469543, 5.0, 0.3817440867424011, 'Severe', 'Severe', 125)

('B4030', 5.0, 0.6705318093299866, 'Undef', 0, 'Severe', 'Undef', 126)

changing KEEP from: Severe to Mild with conf: 0.5442060828208923
('B4030', 1.0, 0.5442060828208923, 'Undef', 0, 'Mild', 'Undef', 127)

('B4030', 1.0, 0.6608970761299133, 'Undef', 0, 'Mild', 'Undef', 128)
For Severe conf: 0.606462299823761/0.2859248220920563
Mild

changing KEEP from: Mild to Severe with conf: 0.606462299823761
('B4030', 5.0, 0.606462299823761, 1.0, 0.2859248220920563, 'Severe', 'Mild', 129)
For Mild conf: 0.5470058917999268/0.3643666207790375
Mild

changing KEEP from: Severe to Mild with conf: 0.5470058917999268
('B4030', 1.0, 0.5470058917999268, 1.0, 0.3643666207790375, 'Mild', 'Mild', 130)

('B4030', 1.0, 0.33054420351982117, 'Undef', 0, 'Mild', 'Undef', 131)
number of frames :  131  for this video
Total cnf is  60.92518290877342
ONE ROW not implemented
D

In [ ]:
print ("clean RESULT DIR")
RDIR=os.path.join ("/content/RESULT/")
LRD=os.listdir (RDIR)
print (len (LRD), " files found")
xx=input ("Delet =cr")
nd=0
if xx=="":
  print ("deleting")
  for fl in LRD:
    fp=os.path.join (RDIR, fl)
    os.remove(fp)
    nd+=1
  print ("removed ", nd, "files")

clean RESULT DIR
20  files found
Delet =crno


In [ ]:
print ("zip results")
RP=os.path.join ("/content/RESULT")
RL=os.listdir(RP)
print (len(RL), " rsults found in", RP)
xx=input ("compress to STORE cr=yes")
if xx=="":
  zip_name = "CP_RESULTS_r9_v36.zip"
  !zip -r /content/STORE/CP_RESULTS__r9_v36.zip /content/RESULT

zip results
20  rsults found in /content/RESULT
compress to STORE cr=yesno


In [ ]:
print (" move to docs")
xx= input ("move to mydrive-colab_data=RESULTS_YOLO y=cr")
if xx=="":
  DOC_P=os.path.join ("/content/drive/MyDrive/Colab_Data/RESULTS_YOLO")
  !cp  /content/STORE/CP_RESULTS__r9_v36.zip    /content/drive/MyDrive/Colab_Data/RESULTS_YOLO
  print ("CP_RESULTS__r9_v36.zip" , " moved to google docs at", DOC_P)

 move to docs
move to mydrive-colab_data=RESULTS_YOLO y=crno


# New Section